In [1]:
from keras import backend as K
from keras.models import load_model
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils_impl import predict_signature_def
import tensorflow as tf
from tensorflow.contrib.session_bundle import exporter
import keras as k
import os

/Users/markkeinhorster/git/tensorflow_serving_weather_service/tf_weather_service/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
tf.__version__

'1.8.0'

In [3]:
model_version = 1
export_path = os.path.join("./models/mnist",str(model_version))
print export_path

./models/mnist/1


In [4]:
model = load_model('mnist_pred.h5')
tf_builder = builder.SavedModelBuilder(export_path)
tensor_info_input = tf.saved_model.utils.build_tensor_info(model.input)
tensor_info_output = tf.saved_model.utils.build_tensor_info(model.output)

prediction_signature = (
      tf.saved_model.signature_def_utils.build_signature_def(
          inputs={'input': tensor_info_input},
          outputs={'prediction': tensor_info_output},
          method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME))
    
legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')
with K.get_session() as sess:
    tf_builder.add_meta_graph_and_variables(
        sess=sess,
        tags=[tag_constants.SERVING],
        clear_devices = True,
        signature_def_map={
            signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: prediction_signature},
        legacy_init_op=legacy_init_op
    )
    tf_builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ./models/mnist/1/saved_model.pb
